![title](https://i.ibb.co/f2W87Fg/logo2020.png)

---


<table  class="tfo-notebook-buttons" align="left"><tr><td>
    
<a href="https://colab.research.google.com/github/adf-telkomuniv/CV2020_Exercises/blob/master/CV2020 - 04 - TensorFlow.ipynb" source="blank" ><img src="https://colab.research.google.com/assets/colab-badge.svg"></a>
</td><td>
<a href="https://github.com/adf-telkomuniv/CV2020_Exercises/blob/master/CV2020 - 04 - TensorFlow.ipynb" source="blank" ><img src="https://i.ibb.co/6NxqGSF/pinpng-com-github-logo-png-small.png"></a>
    
</td></tr></table>


# Task 4 - TensorFlow


You've written a lot of code in this assignment to provide a whole host of neural network functionality. Dropout, Batch Norm, and 2D convolutions are some of the workhorses of deep learning in computer vision. 

You've also worked hard to make your code efficient and vectorized.

For the this assignment, though, we're going to leave behind your beautiful codebase and instead migrate to one of two popular deep learning frameworks: in this instance, **TensorFlow**

The goals of this assignment are as follows:

    * Use TensorFlow at three different levels of abstraction,
    * Barebone TensorFlow: work directly with low-level TensorFlow graphs.
    * Keras Sequential API: use tf.keras.Sequential to define a linear feed-forward network.
    * Keras Model API: use tf.keras.Model to define arbitrary neural network architecture.



Write down your Name and Student ID

In [ ]:
## --- start your code here ----

NIM = ??
Nama = ??

## --- end your code here ----

---
## About Tensorflow


<img src="https://www.gstatic.com/devrel-devsite/va3a0eb1ff00a004a87e2f93101f27917d794beecfd23556fc6d8627bba2ff3cf/tensorflow/images/lockup.svg" alt="tensorflow" width="300px"/>

[TensorFlow](https://www.tensorflow.org/) is a **Deep Learning Library**, developed by the Google Brain Team within the Google Machine Learning Intelligence research organization, for the purposes of machine learning and artificial neural network research.

TensorFlow is a system for executing computational graphs over Tensor objects, with native support for performing backpropogation for its Variables. In it, we work with Tensors which are n-dimensional arrays analogous to the numpy ndarray.



**Tensorflow Key Features**

* Define, optimize and efficiently calculate mathematical expressions involving multi-dimensional arrays (tensors).
* Programming support from deep neural networks and machine learning techniques.
* Use of GPU computing and automatic memory optimization.
* High computing capability across machines and large data sets.

TensorFlow is available with Python and C ++ support, but the Python API is more supported and easier to learn.




---
## About Keras

<img src="https://s3.amazonaws.com/keras.io/img/keras-logo-2018-large-1200.png" alt="keras" width="300px"/>

[Keras](https://keras.io/) is a very modular and minimalist **Deep Learning Library**, written in Python and capable of running on TensorFlow or Theano. This library was developed with a focus on enabling fast experiments.

At first Keras was developed to help users to easily use Theano and Tensorflow's which at the time was very technical and complex in implementation.

Since Tensorflow version 1.5, Keras was adopted by Google and since then the built API has been included in the Tensorflow distribution.

---

Working with Tensorflow will give us benefits:

* Our code will now run on GPUs! Much faster training. Writing your own modules to run on GPUs is beyond the scope of this class, unfortunately.

* We want you to be ready to use one of these frameworks for your project so you can experiment more efficiently than if you were writing every feature you want to use by hand. 

* We want you to stand on the shoulders of giants! TensorFlow and PyTorch are both excellent frameworks that will make your lives a lot easier, and now that you understand their guts, you are free to use them :) 

* We want you to be exposed to the sort of deep learning code you might run into in academia or industry. 

---
## GPU Runtime
Since we're going to use TensorFlow, we can utilize the GPU to accelerate the process

For that, make sure that this Colaboratory file is set to use GPU

* select **Runtime** in taskbar
* select **Change Runtime Type**
* choose Hardware accelerator **GPU**

<center>
  
![gpu](https://i.ibb.co/QX3Brf0/gpu.png)


---
---
#[Part 0] Import Libraries and Load Data

---
## 1 - Import Libraries

Import required libraries

In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import os, datetime
from tabulate import tabulate

%matplotlib inline
%load_ext autoreload
%load_ext tensorboard
%autoreload 2

# tensorboard log
logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))

---
## 2 - Load CIFAR-10

In [ ]:
(X_train_ori, y_train), (X_test_ori, y_test) = tf.keras.datasets.cifar10.load_data()

class_names = ['plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']


---
## 3 - Split Validation Data

In [ ]:
X_val_ori = X_train_ori[-10000:,:]
y_val     = y_train[-10000:]

X_train_ori = X_train_ori[:-10000, :]
y_train     = y_train[:-10000]

---
## 4 - Normalize and Reshape Data

In [ ]:
X_train = X_train_ori.astype('float32')
X_val   = X_val_ori.astype('float32')
X_test  = X_test_ori.astype('float32')

mean_pixel = X_train.mean(axis=(0, 1, 2), keepdims=True)
std_pixel = X_train.std(axis=(0, 1, 2), keepdims=True)

X_train = (X_train - mean_pixel) / std_pixel
X_val   = (X_val - mean_pixel) / std_pixel
X_test  = (X_test - mean_pixel) / std_pixel

X_train = X_train.astype('float32')
X_val   = X_val.astype('float32')
X_test  = X_test.astype('float32')


print('X_train.shape =',X_train.shape)
print('X_val.shape   =',X_val.shape)
print('X_test.shape  =',X_test.shape)

y_train = y_train.ravel()
y_val   = y_val.ravel()
y_test  = y_test.ravel()

print('\ny_train.shape =',y_train.shape)
print('y_val.shape   =',y_val.shape)
print('y_test.shape  =',y_test.shape)

---
---
# [Part 1] Keras Model API
Implementing a neural network using the low-level TensorFlow API is a good way to understand how TensorFlow works, but it's a little inconvenient, we had to manually keep track of all Tensors holding learnable parameters, and we had to use a control dependency to implement the gradient descent update step. .

Furthermore, since the release of version 2.0, Tensorflow is standardizing its implementation using High-level Keras API. This is enabled by the introduction of the ["Eager Execution mode"](https://www.tensorflow.org/guide/eager) which allows you to evaluates Tensor operations imperatively (in the order you write them), similar to NumPy and PyTorch, without building graphs.

Eager-mode is slightly less performant but a lot more intuitive. This makes it easy to get started with TensorFlow and debug models, and it reduces boilerplate as well.


In this part of the notebook we will define neural network models using the&nbsp; `tf.keras.Sequential` &nbsp;and&nbsp; `tf.keras.Model` API. 


---
## 1 - Keras Model and Layers

Keras is a Deep Learning API that was built as an independent open source project by more than 700 contributors. 

During its construction until, Keras is constantly updated so that there are often changes in the technical side of writing code. One of them is how to define a network model.

There are two basic model building in Keras, using linear [**Sequential**](https://www.tensorflow.org/api_docs/python/tf/keras/Sequential) model, or using more advance graphical [**Models**](https://www.tensorflow.org/api_docs/python/tf/keras/Model).

For now, let's import those two packages


In [ ]:
from tensorflow.keras import Sequential
from tensorflow.keras import Model


For this example, let's create a **2 layers neural network** with **100 neurons** in its hidden layer

There are four types of layers that we will use to build this model:

     * Input layer      to receive input shape
     * Flatten layer    to reshape input into one-dimensional matrix for neural network
     * Dense layer      to add affine fully connected layer
     * Activation layer to add nonlinearity
     
For other types of layers, you can look it in **[tf.keras.layers](https://www.tensorflow.org/api_docs/python/tf/keras/layers)**

In [ ]:
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Activation

---
## 2 - Old Sequential API

The first way to build a model using Keras, and one of the oldest ways, is to initialize the [**Sequential**](https://www.tensorflow.org/api_docs/python/tf/keras/Sequential) model object, 

then one by one we add the layers that we want to stack as follows


In [ ]:
model = Sequential(name='my_model_1')

model.add( Input((32,32,3)) )           # input layer to receive image 32x32x3
model.add( Flatten() )                  # Flatten layer to reshape input into 3072x1
model.add( Dense(100) )                 # First affine layer (hidden layer) with 100 neurons
model.add( Activation('sigmoid') )      # Sigmoid activation function
model.add( Dense(10) )                  # Second affine layer (output layer) with 10 neuron
model.add( Activation('softmax') )      # Softmax activation function



The following summary shows how many parameters each layer is made up of (the number of entries in the weight matrices and bias vectors). Note that a value of ```None``` in a particular dimension of a shape means that the shape will dynamically adapt based on the shape of the inputs.

In [ ]:
model.summary()

---
## 3 - Compact Sequential API

The update on the Keras layer API allows us to add the input shape into the first layer directly.
It also allows up to add the activation function directly from the `Dense` layer without adding the `Activation` layer. 


In [ ]:
# create model new
model = Sequential(name='my_model_2')

model.add( Flatten(input_shape=(32,32,3))   )    # Flatten layer to receive image 32x32x3 and reshape it into 3072x1
model.add( Dense(100, activation='sigmoid') )    # First affine layer (hidden layer) with 100 neurons and sigmoid activation
model.add( Dense(10,  activation='softmax') )    # Second affine layer (output layer) with 10 neuron and softmax activation
      
model.summary()

---
## 4 - Functional Model API
The third way to build models is to use **functional API** which allows us to build more complex model graphs, for example, having many input and output. The functional API can handle models with non-linear topology, models with shared layers, and models with multiple inputs or outputs. After the graph is defined, group layers into an object using [**Models**](https://www.tensorflow.org/api_docs/python/tf/keras/Model) module

The following is an example of building a model using **functional API**

In [ ]:

# create model graph
in_node  = Input(shape=(32,32,3))                 # define input node to receive image 32x32x3
x        = Flatten() (in_node)                    # define x node as Flatten layer that receive input node
x        = Dense(100, activation='sigmoid')(x)    # pass x node to Dense hidden layer
out_node = Dense(10,  activation='softmax')(x)    # define output node as Dense layer that receive x node

# initialize the model
model = Model(in_node, out_node, name='my_model_3')

model.summary()


---
## 5 - Sequential API (Current)
The fourth way is a new way, and the current standard, of building a Sequential model. 

Similar to the first and second ways, but we can directly register the layers in the list when initializing the Sequential object as follows

In [ ]:
# create model compact sequential

model = Sequential([                    
    Flatten(input_shape=(32,32,3)),     # Flatten layer to receive image 32x32x3 and reshape it into 3072x1
    Dense(100, activation='sigmoid'),   # First affine layer (hidden layer) with 100 neurons and sigmoid activation  
    Dense(10,  activation='softmax')    # Second affine layer (output layer) with 10 neuron and softmax activation
], name='my_model_4')


model.summary()

The four models defined above are the same model. You can see it from the summaries that all model has $308,310$ total parameters (weights)

---
## 6 - Plot Model

Plot model is a utility function to converts a Keras model to dot format and save to a file. This enables us to visualize the model more easily, especially for non-linear models.

In [ ]:
from tensorflow.keras.utils import plot_model

plot_model(model, 
           to_file=model.name+'.png', 
           show_shapes=True, 
           show_layer_names=False,
           rankdir='LR',
           dpi=70
          )


---
---
# [Part 2] Activation and Optimizer 

---
## 1 - Activation Functions

Activation functions are a core ingredient in deep neural networks. In fact they are what allows us to make use of multiple layers in a neural network. There are a number of different activation functions, each of which are more or less useful under different circumstances. The four activation functions that you are most likely to encounter are, arguably, [ReLU](https://www.tensorflow.org/api_docs/python/tf/keras/layers/ReLU), [Tanh](https://www.tensorflow.org/api_docs/python/tf/keras/activations/tanh), [Sigmoid](https://www.tensorflow.org/api_docs/python/tf/keras/activations/sigmoid), and [Softmax](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Softmax). 

More of activation functions can be seen at [keras activations](https://keras.io/api/layers/activations/) or [tf.keras activations](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Activation)

---
## 2 - Loss Functions

To optimize the model, we need to specify a loss function for our classifier. This tells us how good our model's predictions are compared to the actual labels (the targets), with a lower loss meaning better predictions. There are also various types of **loss functions** for various cases such as:
* `categorical crossentropy` &nbsp; for multi-class classifications
* `binary crossentropy` &nbsp; for binary classification
* `mean squared error` &nbsp; for regression
* and many others

<br>

The standard loss function to use with a multi-class classifier is the **cross-entropy loss** also known as the "negative log likelihood" for a classifier.

For a a classification problem with $C$ classes, the cross-entropy loss is defined as

$$Loss = -\frac{1}{N}\sum_{i=1}^N \sum_{c=1}^C log( p(y|X_i)_c) \mathbb{1}[y_i=c]$$



More of loss functions can be seen at [keras losses](https://keras.io/api/losses/) or [tf.keras losses](https://www.tensorflow.org/api_docs/python/tf/keras/losses)

---
## 3 - Optimizer Functions

After determining the loss function, we can choose the optimization function that we will use to train the model. The optimizer also responsible for controlling the learning rate.

There are various types of optimization functions such as:
* `sgd` &nbsp;for standard&nbsp; `stochastic gradient descent`, including &nbsp;`nesterov`
* `rmsprop` &nbsp;which is a further development of 'sgd'
* `adam` &nbsp;as the current standard optimization function
* and many others

<br>
Here is an illustration showing how a few of these methods perform on a toy problem: 

<table>
  <tr><td  align="center">
    <img src="https://cs231n.github.io/assets/nn3/opt1.gif" width="70%" 
         alt="Optimization algorithms visualized over time in 3D space.">
  </td></tr>
  <tr><td align="center">
    <b>Figure:</b> Optimization algorithms visualized over time in 3D space.<br/>(Source: <a href="http://cs231n.github.io/neural-networks-3/">Stanford class CS231n</a>, MIT License, Image credit: <a href="https://twitter.com/alecrad">Alec Radford</a>)
  </td></tr>
</table>


More of optimizer functions can be seen at [keras optimizers](https://keras.io/api/optimizers/) or [tf.keras optimizers](https://www.tensorflow.org/api_docs/python/tf/keras/optimizers)

---
---
# [Part 3] Preparing Data with TensorFlow
At the moment, our training data consists of two large tensors. The images are stored in a tensor of shape $[40000, 32, 32, 3]$, consisting of all the $32 \times 32 \times 3$ images matrices stacked together. The labels are stored in a 1D vector of shape $[40000]$. 

We wish to train a model using **mini-batch stochastic gradient descent**. In order to do so, we need to shuffle the data and split it into smaller (mini-)batches. We also convert the data from numpy arrays to TensorFlow Tensors.

In order to do this batching (and shuffling) we will use the Tensorflow [Dataset API](https://www.tensorflow.org/api_docs/python/tf/data/Dataset), which is a set of simple reusable components that allow you to build efficient data pipelines. 

---
## 1 - Define Batch Dataset

We start by defining the `batch_size` hyperparameter of our model. 
> This hyperparameter controls the sizes of the mini-batches (chunks) of data that we use to train the model. The value you use will affect the memory usage, speed of convergence and potentially also the performance of the model. It also interacts with the learning rate used in gradient descent.  

Then, we  group the image and label tensors together into a tuple and then split them into individual entries using [from_tensor_slices()](https://www.tensorflow.org/api_docs/python/tf/data/Dataset#from_tensor_slices) function.

The next thing we do is add a shuffle component and put the batch component to return a random batch of slices from the dataset. The output of this pipeline will be tuples of tensors containing images and label. The images will be of shape `(batch_size, 32, 32, 3)` and the labels of shape `(batch_size, )`

In [ ]:
# define the batch size
batch_size = 128

train_ds = tf.data.Dataset.from_tensor_slices((X_train, y_train))
train_ds = train_ds.shuffle(buffer_size=batch_size * 10)
train_ds = train_ds.batch(batch_size)

image_shape = X_train.shape[1:]

We do the same for the validation dataset, except we don't need to shuffle this time.

In [ ]:
val_ds = tf.data.Dataset.from_tensor_slices((X_val, y_val))
val_ds = val_ds.batch(batch_size)

---
---
# [Part 4] Tensorflow Training Function

In graph mode, TensorFlow builds up a "computation graph" which captures all operations of the model and their dependencies. For training, the gradient can then be computed by traversing backwards from every node through its dependents and applying the "chain rule" of differentiation.

However, in Eager mode, we don't have the concept of the computation graph anymore. Operations are performed imperatively (in the order in which they were executed). 


---
## 1 - Three-layer Neural Net

In this section we'll build a classifier. Here you will complete the implementation of Three-layer Neural Net with the following architecture:

<pre>
|              |              |         |
| <font color='red'>dense</font> - <font color=''>relu</font> | <font color='red'>dense</font> - <font color=''>relu</font> |  <font color='brown'>dense</font>  |
|              |              |         |
|       1      |      2       |    3    |
</pre>

#### <font color='red'>**EXERCISE:** </font>
Define your classification model. 

You can define the model using Sequential or Functional API. You can also experiment by trying to change the layer type, activation functions, and number of neurons. However, for this pipeline, **do not** add activation function to the output layer.

In [ ]:
model = Sequential([
                             
    Flatten(input_shape=(32,32,3)),  

    ??,   # add a dense layer with 256 neurons and relu activation
    ??,   # add a dense layer with 128 neurons and relu activation

    # Create an "output layer" with 10 neurons, 
    # do not add activation to this output layer
    Dense(10),

], name='my_model')


Now display the architecture summary

In [ ]:
model.summary()

**EXPECTED OUTPUT**:
<pre>
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
=================================================================
flatten_? (Flatten)          (None, 3072)              0         
_________________________________________________________________
dense_? (Dense)              (None, 256)               786688    
_________________________________________________________________
dense_? (Dense)              (None, 128)               32896     
_________________________________________________________________
dense_? (Dense)              (None, 10)                1290      
=================================================================
Total params: 820,874
Trainable params: 820,874
Non-trainable params: 0

let's also plot the model design

In [ ]:
plot_model(model, 
           to_file=model.name+'.png', 
           show_shapes=True, 
           show_layer_names=False,
           rankdir='LR',
           dpi=70
          )

**EXPECTED OUTPUT**:

<img src="https://i.ibb.co/tK9yMJY/3layernet.png" width="80%">

---
## 2 - Define Optimizer and Loss

Here we use the **Adam optimizer** to train our neural networks. Adam is a variant of stochastic gradient descent which often performs well in practice. 

#### <font color='red'>**EXERCISE:** </font>
Define  the oprimizer

use ```tf.keras.optimizers.Adam()```.

In [ ]:
# Instantiate an optimizer to train the model.
optimizer = ??


#### <font color='red'>**EXERCISE:** </font>
Define  the loss

use ```tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)```.

In [ ]:
# Instantiate a loss function.
loss_fn = ??


---
## 3 - Define Metrics

Select metrics to measure the loss and the accuracy of the model. 
These metrics accumulate the values over epochs and then print the overall result.

In [ ]:
from tensorflow.keras import metrics

train_loss     = metrics.Mean(name='train_loss')
val_loss       = metrics.Mean(name='val_loss')

train_accuracy = metrics.SparseCategoricalAccuracy(name='train_accuracy')
val_accuracy   = metrics.SparseCategoricalAccuracy(name='val_accuracy')

---
## 4 - Train Step Function


The default runtime in TensorFlow 2.0 is eager execution. TensorFlow uses a mechanism called the **"GradientTape"** for computing gradients in Eager mode. 

Basically, the gradient tape records the order of all operations as they are executed, and can then be "run backwards" (traversed from last to first operation) for computing the gradients.

This is great for debugging, but graph compilation has a definite performance advantage. 

Decribing your computation as a static graph enables the framework to apply global performance optimizations.


#### <font color='red'>**EXERCISE:** </font>
Define a function to train the model using `tf.GradientTape`. This opens a GradientTape to record the operations run during the forward pass, which enables auto-differentiation. Inside the function we run the forward pass of the layer, and update the parameters using the recorded gradients of the trainable variables.


In [ ]:
@tf.function
def train_step(x, y):

    # Initialise a GradientTape to track the operations
    with tf.GradientTape() as tape:

        # Compute the logits (un-normalised scores) of the current batch of examples
        # using the neural network architecture we defined earlier
        # call model() function with input x and set training=True
        logits = ??

        # Compute the loss value for this minibatch.
        # call loss_fn() function with input y and logits
        loss_value = ??

    # Use the gradient tape to automatically retrieve
    # the gradients of the trainable variables with respect to the loss.
    gradients = tape.gradient(loss_value, model.trainable_weights)

    # Run one step of gradient descent by updating
    # the value of the variables to minimize the loss.
    optimizer.apply_gradients(zip(gradients, model.trainable_weights))

    # Update train metrics
    train_loss(loss_value)
    train_accuracy.update_state(y, logits)
    


---
## 5 - Test Step Function

Define a function to evaluate the model. This function will run a validation loop at the end of each epoch.

#### <font color='red'>**EXERCISE:** </font>
complete the function


In [ ]:
@tf.function
def test_step(x, y):

    # call model() with input x and set training=False
    logits = ??

    # call loss_fn() function with input y and logits
    loss_value = ??

    # Update val metrics
    val_loss(loss_value)
    val_accuracy.update_state(y, logits)

    return logits, loss_value, val_accuracy.result()


---
## 6 - Training Loop

Define the training loop as follow:
* Define a `for` loop that iterates over epochs. For each epoch:
    * open a `for` loop that iterates over the training dataset, in batches.
      <br>for each train batch, call the `train_step()` function
    * open another `for` loop that iterates over the validation dataset, in batches.
      <br>for each validation batch, call the `test_step()` function


#### <font color='red'>**EXERCISE:** </font>
complete the function<br>
and train the network for 20 epochs

In [ ]:
# The number of epochs to run
num_epochs = 20  

# Lists to store the loss and accuracy of every epoch
history = {}
history['loss']     = []
history['val_loss'] = []
history['acc']      = []
history['val_acc']  = []

for epoch in range(num_epochs):

    # Loop over our data pipeline
    for x_batch_train, y_batch_train in train_ds:
        # call train_step() function with input x_batch_train and y_batch_train
        ??
        
    # Run a validation loop at the end of each epoch.
    for x_batch_val, y_batch_val in val_ds:
        # call test_step() function with input x_batch_val and y_batch_val
        ??
    
    # add current loss and accuracy to history
    history['loss'].append(train_loss.result())
    history['val_loss'].append(val_loss.result())

    history['acc'].append(train_accuracy.result())
    history['val_acc'].append(val_accuracy.result())

    # print current loss and accuracy
    template = 'Epoch {:03d}, Loss: {:.4f}, Train Acc: {:.2%}, Val Acc: {:.2%}'
    print(template.format(epoch+1,
                          train_loss.result(), 
                          train_accuracy.result(),
                          val_accuracy.result()))
    

    # Reset training metrics at the end of each epoch
    train_accuracy.reset_states()
    val_accuracy.reset_states()


**EXPECTED OUTPUT**:

<pre>The loss should start from ~1.7 and end around 0.9 with validation accuracy around 49%

The code block shows a typical training loop. There's actually an easier way to do this using Tensorflow and Keras, which we'll see later.

---
## 7 - Visualize Training History
Plot the loss and accuracy during training

In [ ]:
plt.rcParams['figure.figsize'] = [14, 3.5]
plt.subplots_adjust(wspace=0.2)

plt.subplot(121)
# Plot training & validation accuracy values
plt.plot(history['acc'])
plt.plot(history['val_acc'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Val'])

plt.subplot(122)
# Plot training loss values
plt.plot(history['loss'])
plt.plot(history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Val'])
plt.show()

**EXPECTED OUTPUT**:

<pre>You should see overfitting occured

You should see that if we only use ordinary Artificial Neural Networks, overfitting occurs during training.

You can improve this using more advanced architecture such as Convolutional Neural Network. 

You can also improve the model by adding more advanced optimization scheme such as adding Dropout Layer, Batch Normalization Layer, Regularizers, and so on.

> For Dropout, see [Tensorflow documentation](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Dropout) and [research paper](https://www.cs.toronto.edu/~hinton/absps/JMLRdropout.pdf) 

> For Batch Normalization, see [Tensorflow documentation](https://www.tensorflow.org/api_docs/python/tf/keras/layers/BatchNormalization) and [research paper](http://proceedings.mlr.press/v37/ioffe15.pdf)

---
## 8 - Evaluate Model

Now let's test the model to the unseen dataset.

First, convert testing dataset and its targets into tensor

In [ ]:
tf_X_test = tf.convert_to_tensor(X_test, dtype=tf.float32)
tf_y_test = tf.convert_to_tensor(y_test, dtype=tf.int32)

Then call the test_step function, and get the output tuple containing logits, loss, and accuracy

we should get accuracy around `48%`

In [ ]:
test_logits, test_loss, test_accuracy = test_step(tf_X_test, tf_y_test)

# get the predicted class
y_pred = tf.argmax(test_logits, axis=1, output_type=tf.int32)

print('Completed testing on', tf_X_test.shape[0], 'examples...')
print('Loss: {:.3f}, Accuracy: {:.3%}'.format(test_loss, test_accuracy))

**EXPECTED OUTPUT**:

<pre>You should see testing accuracy around 48%

---
## 9 - View Result
Now to visualize some of the model's predictions:

In [ ]:
fig, ax = plt.subplots(2,10,figsize=(22,6))
fig.subplots_adjust(hspace=0.1, wspace=0.1)
label = y_test.ravel()
for j in range(0,2):
    for i in range(0, 10):

        img_index = np.random.randint(0, 10000)
        ax[j,i].imshow(X_test_ori[img_index])

        actual_label    = int(y_test[img_index])
        predicted_label = int(y_pred[img_index])

        color = 'red'
        if actual_label == predicted_label:
            color = 'green'

        ax[j,i].set_title("Actual: {} ({})\n Predicted: {} ({})".format(
            actual_label, class_names[actual_label], predicted_label, class_names[predicted_label]
            ), color=color)
        ax[j,i].axis('off')
plt.show()

Run the cell above again to see another set of examples

---
---
# [Part 5] Keras API Training Function

---
## 1 - One-Hot Matrix

Up until now, we're using the `sparse categorical cross-entropy` loss since our targets are integers (index of the labels). Depending on your dataset, you might find dataset with one-hot encoded tagets.

One-hot representations are usually good for categorical classes (multi-class problems) where no such ordinal relationship exists. In this case, a one-hot encoding can be applied to label representation as it has clear interpretation and its values are separated each is in different dimension.

It is more intuitive if you implement the softmax loss from scratch. However, using one-hot representation may slow down the training when you have hundreds of classes to classify.

For that, the first step that we have to do is convert the target into what is known as one-hot matrix. It change the target label into a sparse matrix with size of class number, with one in the index of the label and zeros in everywhere else. 

With Keras, we can use **to_categorical** functions from **tf.keras.utils** module. And when we're using one-hot representations as target, we can use `categorical cross-entropy` for the loss.

---
#### <font color='red'>**EXERCISE:** </font>

Change target vector **y_train**, **y_val**, and **y_test** each into a One-Hot Matrix using **to_categorical** method

In [ ]:
from tensorflow.keras.utils import to_categorical

y_train_hot = to_categorical(y_train, 10)
y_val_hot   = ??
y_test_hot  = ??


Check your implementation

In [ ]:
print('y_train_hot.shape =',y_train_hot.shape)
print('y_val_hot.shape   =',y_val_hot.shape)
print('y_test_hot.shape  =',y_test_hot.shape)

**EXPECTED OUTPUT**:
<pre>
 y_train_hot.shape = (40000, 10)
 y_val_hot.shape   = (10000, 10)
 y_test_hot.shape  = (10000, 10)

example of 10 first one-hot label from training data

In [ ]:
print('  |     |  class:\ni |  y  |  0 1 2 3 4 5 6 7 8 9\n---------------------------------')
for i in range(10):
    print(i, '|', y_train[i], '|', y_train_hot[i,:].astype('int'))

**EXPECTED OUTPUT**:
<pre>
  |   |  class:
i | y |  0 1 2 3 4 5 6 7 8 9
-------------------------------
0 | 6 | [0 0 0 0 0 0 1 0 0 0]
1 | 9 | [0 0 0 0 0 0 0 0 0 1]
2 | 9 | [0 0 0 0 0 0 0 0 0 1]
3 | 4 | [0 0 0 0 1 0 0 0 0 0]
4 | 1 | [0 1 0 0 0 0 0 0 0 0]
5 | 1 | [0 1 0 0 0 0 0 0 0 0]
6 | 2 | [0 0 1 0 0 0 0 0 0 0]
7 | 7 | [0 0 0 0 0 0 0 1 0 0]
8 | 8 | [0 0 0 0 0 0 0 0 1 0]
9 | 3 | [0 0 0 1 0 0 0 0 0 0]

---
## 2 - Three-layer Neural Net

Same as before, we define the classifier model of Three-layer Neural Net with the following architecture:

<pre>
|              |              |                 |
| <font color='red'>dense</font> - <font color=''>relu</font> | <font color='red'>dense</font> - <font color=''>relu</font> | <font color='brown'>dense</font> - <font color=''>softmax</font> |
|              |              |                 |
|       1      |      2       |        3        |
</pre>
<br>


You can define the model using Sequential or Functional API. You can also experiment by trying to change the layer type, activation functions, and number of neurons. 

For this model, use `softmax` activation for the output layer.

#### <font color='red'>**EXERCISE:** </font>
Define your classification model. 


In [ ]:
model = Sequential([
                             
    Flatten(input_shape=(32,32,3)),  

    ??,   # add a dense layer with 256 neurons and relu activation
    ??,   # add a dense layer with 128 neurons and relu activation
    ??    # add a dense layer with 10 neurons and softmax activation

], name='my_model')


Show the model summary

In [ ]:
model.summary()

**EXPECTED OUTPUT**:
<pre>
Layer (type)                 Output Shape              Param #   
=================================================================
flatten_? (Flatten)          (None, 3072)              0         
_________________________________________________________________
dense_? (Dense)              (None, 256)               786688    
_________________________________________________________________
dense_? (Dense)              (None, 128)               32896     
_________________________________________________________________
dense_? (Dense)              (None, 10)                1290      
=================================================================
Total params: 820,874
Trainable params: 820,874
Non-trainable params: 0
_________________________________________________________________

let's also plot the model design

In [ ]:
plot_model(model, 
           to_file=model.name+'.png', 
           show_shapes=True, 
           show_layer_names=False,
           rankdir='LR',
           dpi=70
          )

**EXPECTED OUTPUT**:

<img src="https://i.ibb.co/tK9yMJY/3layernet.png" width="80%">

---
## 3 - Compile Model

When using the Keras API, to register loss functions and its optimization function, we use `.compile()` method. Conveniently, we can choose the loss, optimizer, and metrics using their string identifier.

#### <font color='red'>**EXERCISE:** </font>
compile your model using 
* `'categorical_crossentropy'` loss, 
* `'adam'` optimizer, and 
* `['accuracy']` metrics



In [ ]:
# Compile model
model.??


To see the string identifier for loss functions, see [keras losses](https://keras.io/api/losses/) or [tf.keras losses](https://www.tensorflow.org/api_docs/python/tf/keras/losses)

To see the string identifier for optimizers, see [keras optimizers](https://keras.io/api/optimizers/) or [tf.keras optimizers](https://www.tensorflow.org/api_docs/python/tf/keras/optimizers)

---
## 4 - Train Model

Now we can train the model by calling the &nbsp; `.fit()` &nbsp; method.

Run the training process for  20 epochs with batch size=128

In [ ]:
num_epochs = 20
batch_size = 128

tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)

history = model.fit(X_train, y_train_hot, 
                    validation_data=(X_val, y_val_hot),
                    epochs=num_epochs, 
                    batch_size=batch_size, 
                    callbacks = [tensorboard_callback],
                    verbose=2)

**EXPECTED OUTPUT**:

<pre>You should get similar result from before
The loss should start from ~1.7 and end around 0.5 with validation accuracy around 49%

you can further train the model simply by re-run the cell 

---
## 5 - Visualize Training History

Visualize the train-validation accuracy

In [ ]:
plt.rcParams['figure.figsize'] = [14, 3.5]
plt.subplots_adjust(wspace=0.2)

plt.subplot(121)
# Plot training & validation accuracy values
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'])

plt.subplot(122)
# Plot training & validation loss values
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'])
plt.show()

**EXPECTED OUTPUT**:

<pre>You should see overfitting occured

We can also visualize the training history to Tensorboard

In [ ]:
%tensorboard --logdir logs

---
## 6 - Evaluate Model
Next, let's evaluate the accuracy of the models that have been trained

we should get accuracy around `48%`

In [ ]:
scores = model.evaluate(X_test, y_test_hot, verbose=1)
print("\nModel Accuracy: %.2f%%" % (scores[1]*100))

---
## 7 - View Result
Now to visualize some of the model's predictions:

In [ ]:
y_pred = model.predict(X_test, verbose=0)
y_pred = np.argmax(y_pred, axis=1)

In [ ]:
fig, ax = plt.subplots(2,10,figsize=(22,6))
fig.subplots_adjust(hspace=0.1, wspace=0.1)

for j in range(0,2):
    for i in range(0, 10):

        img_index = np.random.randint(0, 10000)
        ax[j,i].imshow(X_test_ori[img_index])

        actual_label    = int(y_test[img_index])
        predicted_label = int(y_pred[img_index])

        color = 'red'
        if actual_label == predicted_label:
            color = 'green'

        ax[j,i].set_title("Actual: {} ({})\n Predicted: {} ({})".format(
            actual_label, class_names[actual_label], predicted_label, class_names[predicted_label]
            ), color=color)
        ax[j,i].axis('off')
plt.show()

Run the cell above again to see another set of examples

---
## 8 - Save Model

Now as an addition, we can save our model for later use by using `.save()` function. This will save both the architecture code and the weight matrices.

The stored model is ready to use when loaded.

In [ ]:
model.save('my_model.h5')

Alternatively, you can opt to save just the weight matrices by using `.save_weights()` function. This way the saved file is smaller and more compact, but you need the exact matched model arcitecture to load it.

This is typically used when you are deploying a system that already has an earlier version of the model embedded, but then you want to update the weights without actually changing the architecture. For example after you trained it further with new additional dataset to improve the accuracy.

In [ ]:
model.save_weights('my_weights.h5')

---
---
# [Part 6] Keras Three-Layer ConvNet

For this part, let's try to improve the classification performance by changing the model to a 3-layer Convolutional Neural Net



---
## 1 - Define Model

Define a three-layer convnet with the same architecture from Part 1.

You can use any model building style that fits to your taste

A three-layer convolutional network with the following architecture:

<pre>
|                       |                       |         |                 |
| <font color='red'>32 @5x5 Conv2D</font> - <font color=''>relu</font> | <font color='red'>16 @3x3 Conv2D</font> - <font color=''>relu</font> | Flatten | <font color='brown'>Dense</font> - <font color=''>softmax</font> |
|                       |                       |         |                 |
|            1          |           2           |         |        3        |
</pre>
<br>

For the **Convolution Layer**, use the [**tensorflow.keras.layers.Conv2D**](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Conv2D)

---
#### <font color='red'>**EXERCISE:** </font>

    define model:
    * 32 5x5 Conv2D layer with relu activation, padding valid, and input_shape=(32,32,3)
    * 16 3x3 Conv2D layer with relu activation and padding valid
    * Flatten layer
    * Dense layer with 10 neurons and softmax activation



In [ ]:
from tensorflow.keras.layers import Conv2D

myConv = Sequential([
    ??
    ??
    ??
    ??
])


Show the model summary

In [ ]:
myConv.summary()

**Expected Output**:
<pre>
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
=================================================================
conv2d_? (Conv2D)            (None, 28, 28, 32)        2432      
_________________________________________________________________
conv2d_? (Conv2D)            (None, 26, 26, 16)        4624      
_________________________________________________________________
flatten_? (Flatten)          (None, 10816)             0         
_________________________________________________________________
dense_? (Dense)              (None, 10)                108170    
=================================================================
Total params: 115,226
Trainable params: 115,226
Non-trainable params: 0

let's also plot the model design

In [ ]:
plot_model(myConv, 
           to_file=model.name+'.png', 
           show_shapes=True, 
           show_layer_names=False,
           rankdir='LR',
           dpi=70
          )

**EXPECTED OUTPUT**:

<img src="https://i.ibb.co/F8JWX3R/3layerconv.png" width="90%">

---
## 2 - Compile Model

Here we have to compile the model by registering a loss function and its optimization function. You should know that `adam` optimizer is better and converge faster than straight `sgd`, <br>but for the sake of this example, here we use the `sgd` optimizer


In [ ]:
# Compile model
myConv.compile(loss='categorical_crossentropy', optimizer='sgd', metrics=['accuracy'])


---
## 3 - Model Checkpoint Callback

When using Keras, we can add several callback functions to the &nbsp; `.fit()` &nbsp; training function. 

A callback is a set of functions to be applied at given stages of the training procedure. You can use callbacks to get a view on internal states and statistics of the model during training. 

One of the very useful callback functions is **ModelCheckpoint**.

You know that when we train neural networks, training accuracy and validation accuracy should always increase with epoch iteration. But at some point, while training accuracy continues to increase, validation accuracy may go down. Which indicates that the network has **overfit** the data. 

Obviously, the model that we want to use is not the model at the end of training epoch, but the one just before the validation accuracy drops.

We can use the **ModelCheckpoint** to periodically save the model only when validation accuracy is increased

In [ ]:
import os
os.mkdir('./model')

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint

filepath = './model/my_model.h5'

myCheckpoint = ModelCheckpoint(filepath, 
                               monitor='val_accuracy',
                               save_best_only=True,
                              )

---
## 4 - Early Stopping Callback

Another useful callback function is **EarlyStopping**

With this, we can monitor the training, and when the loss or accuracy is no longer improving for several epochs, we can terminate the training session and investigate what happened. 

After that, we can think of what we can do to improve training accuracy, then continue the training.

For this example, we'll monitor the validation loss, and when the loss is not decreasing after 5 epochs, we terminate the training.

Other callback functions can be seen at [tf.keras.callbacks](https://www.tensorflow.org/api_docs/python/tf/keras/callbacks)


In [ ]:
from tensorflow.keras.callbacks import EarlyStopping

myStopping = EarlyStopping(monitor='val_loss',
                          patience=5
                          )

---
## 5 - Live Training Plot Callback

For this part, let's add a custom callback function to perform live training plot between training and validation accuracy

---

<font color='red'>**NOTE**</font>: For some reason, as of June 2020, the tensorflow is removing some model parameter in 2.2.0 version. Which interferes with this callback example. So I kind of *quick-fixed* this so the callback just works. This will be further updated after there's further fix and clarification from Tensorflow
<br> see [Issues](https://github.com/tensorflow/tensorflow/issues/40515)

In [ ]:
from IPython.display import clear_output


def translate_metric(x):
    translations = {'acc': "Accuracy", 'loss': "Log-loss (cost function)"}
    if x in translations:
        return translations[x]
    else:
        return x

class PlotLosses(tf.keras.callbacks.Callback):
    def __init__(self, figsize=None):
        super(PlotLosses, self).__init__()
        self.figsize = figsize

    def on_train_begin(self, logs={}):
        
        #=======================================
        # quick-fix to solve missing param
        # print(self.params)
        self.params['metrics'] = ['loss']
        self.params['do_validation'] = ['true']
        #=======================================

        self.base_metrics = [metric for metric in self.params['metrics'] if not metric.startswith('val_')]
        self.logs = []
        

    def on_epoch_end(self, epoch, logs={}):
        self.logs.append(logs)

        clear_output(wait=True)
        plt.figure(figsize=self.figsize)
        
        for metric_id, metric in enumerate(self.base_metrics):
            plt.subplot(1, len(self.base_metrics), metric_id + 1)
            
            plt.plot(range(1, len(self.logs) + 1),
                     [log[metric] for log in self.logs],
                     label="training")
            if self.params['do_validation']:
                plt.plot(range(1, len(self.logs) + 1),
                         [log['val_' + metric] for log in self.logs],
                         label="validation")
            plt.title(translate_metric(metric))
            plt.xlabel('epoch')
            plt.legend(loc='center right')
        
        plt.tight_layout()
        plt.show();
        
myTrainPlot = PlotLosses(figsize=(10, 4))

---
## 6 - Train Model

Now we can train the model by calling the &nbsp; `fit` &nbsp; function



---
#### <font color='red'>**EXERCISE:** </font>

    Train a three-layer ConvNet for 30 epochs with batch size=100
    

In [ ]:
num_epochs = 30
batch_size = 100

history = myConv.fit(X_train, y_train_hot, 
                     validation_data=(X_val, y_val_hot),
                     epochs=num_epochs, 
                     batch_size=batch_size, 
                     callbacks=[myTrainPlot, myCheckpoint, myStopping],
                     verbose=2)

**EXPECTED OUTPUT**:
<pre>
the training should lasts in about 23 epochs before early stopping callback terminate the iteration,
the training loss should start around 1.8 and end around 0.7
while validation loss start around 1.6 then plateau around 1.1


---
## 7 - Evaluate Model
Next, let's evaluate the accuracy of the models that have been trained

First, load the best model from checkpoint

In [ ]:
from tensorflow.keras.models import load_model

myModel = load_model('./model/my_model.h5')

Evaluate the model using &nbsp; `X_test` &nbsp; and &nbsp; `y_test_hot`

In [ ]:
scores = myModel.evaluate(X_test, y_test_hot, verbose=1)

print("\nModel Accuracy: %.2f%%" % (scores[1]*100))

**EXPECTED OUTPUT**:
<pre>
you should get around 62% of testing accuracy

---
## 8 - Test Model on New Image

For this part, you have to test your model on new image

First of all, search for five images on the Internet, then list the URLs to the code below.

The five images must belong to the 10 CIFAR-10 classes that the model recognizes.

---
#### <font color='red'>**EXERCISE:** </font>

    define five image urls
    one image has been given for an example, you can change it

In [ ]:
!wget -q -O 'data_test_0.jpg' 'https://ichef.bbci.co.uk/news/912/cpsprodpb/160B4/production/_103229209_horsea.png'
!wget -q -O 'data_test_1.jpg' '??'
!wget -q -O 'data_test_2.jpg' '??'
!wget -q -O 'data_test_3.jpg' '??'
!wget -q -O 'data_test_4.jpg' '??'

Run and Recognize the images

In [ ]:
import cv2 as cv
from PIL import Image
class_names = ['plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']


for i in range(5):
    # load and open the image
    new_img = Image.open('data_test_'+str(i)+'.jpg')
    new_img = np.array(new_img)

    # resize into (32,32,3)
    new_img2 = cv.resize(new_img, (32,32), interpolation=cv.INTER_AREA)
    plt.imshow(new_img2)
    plt.axis('off')
    plt.show()

    # preprocess the image using X_train mean and std
    new_img2 = (new_img2 - mean_pixel) / std_pixel

    # predict the class
    pred = myModel.predict(new_img2)
    class_id = np.argmax(pred)
    print('predicted id   :',class_id)
    print('predicted class:', class_names[class_id])
    print('--------------------------------\n\n')

---
---
# [Part 7] CIFAR-10 Open-ended Challenge

In this section you can experiment with whatever ConvNet architecture you'd like on CIFAR-10.

You should experiment with **architectures**, **hyperparameters**, **loss functions**, **regularization**, or anything else you can think of to train a model 

You should achieve <font color='blue' size='5'><b>at least 75% accuracy</b></font> on the **validation** set <font color='red' size='4'><b>within 10-20 epochs</b></font>. 


---
## Some things you can try:
- **Filter size**: Above we used 5x5 and 3x3; is this optimal?

- **Number of filters**: Above we used 16 and 32 filters. Would more or fewer do better?

- **Pooling**: We didn't use any pooling above. Would this improve the model?

- **Normalization**: Would your model be improved with batch normalization, layer normalization, group normalization, or some other normalization strategy?

- **Network architecture**: The ConvNet above has only three layers of trainable parameters. Would a deeper model do better?

- **Global average pooling**: Instead of flattening after the final convolutional layer, would global average pooling do better? This strategy is used for example in Google's Inception network and in Residual Networks.

- **Regularization**: Would some kind of regularization improve performance? Maybe weight decay or dropout?

- **Optimization**: You've seen various advanced optimization function. Maybe changing the optimization using Adam or RMSProp will increase the accuracy?

<br><center>
<font color='red' size='4'><b>--- You must design YOUR OWN Architecture --- <br>
--- And train it from scratch --- </b></font>

---
## Tips for training
For each network architecture that you try, you should tune the learning rate and other hyperparameters. 

When doing this there are a couple important things to keep in mind:

- If the parameters are working well, you should see improvement within a few hundred iterations

- Remember the coarse-to-fine approach for hyperparameter tuning: start by testing a large range of hyperparameters for just a few training iterations to find the combinations of parameters that are working at all.

- Once you have found some sets of parameters that seem to work, search more finely around these parameters. You may need to train for more epochs.

- You should use the validation set for hyperparameter search, and save your test set for evaluating your architecture on the best parameters as selected by the validation set.

<center>
<h2><font color='blue'>--- Go Wild, Have Fun, and Happy Training!  --- </font></h2>

---
## 1 - Define Model

---
#### <font color='red'>**EXERCISE:** </font>

    Design your Convolutional Neural Network Architecture

    

In [ ]:
from tensorflow.keras.layers import *

myModel = ??


myModel.summary()

---
## 2 - Train Model

---
#### <font color='red'>**EXERCISE:** </font>

    Compile the model
    Train the model
    

In [ ]:
# Compile model
myModel.compile(??)

num_epochs = ??
batch_size = ??

history = myModel.fit(??)


In [ ]:
# Save model if needed
myModel.save(??)

---
## 3 - Evaluate Model

---
#### <font color='red'>**EXERCISE:** </font>

    evaluate your model on test set
    

In [ ]:
myModel = load_model(??)

train_scores = myModel.evaluate(X_train, y_train_hot, verbose=1)
val_scores   = myModel.evaluate(X_val, y_val_hot, verbose=1)
test_scores  = myModel.evaluate(X_test, y_test_hot, verbose=1)

print("Training Accuracy  : %.2f%%" % (train_scores[1]*100))
print("Validation Accuracy: %.2f%%" % (val_scores[1]*100))
print("Testing Accuracy  :  %.2f%%" % (test_scores[1]*100))

**EXPECTED OUTPUT**:
<pre>
try to get above 80% of accuracy for train, val, and test set

---
## 4 - Test Model on New Image

Test your model on new image

---
#### <font color='red'>**EXERCISE:** </font>

    define five image urls
    one image has been given for an example, you can change it

In [ ]:
!wget -q -O 'data_test_0.jpg' 'https://ichef.bbci.co.uk/news/912/cpsprodpb/160B4/production/_103229209_horsea.png'
!wget -q -O 'data_test_1.jpg' '??'
!wget -q -O 'data_test_2.jpg' '??'
!wget -q -O 'data_test_3.jpg' '??'
!wget -q -O 'data_test_4.jpg' '??'

Run and Recognize the images

In [ ]:
import cv2 as cv
from PIL import Image
class_names = ['plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']


for i in range(5):
    # load and open the image
    new_img = Image.open('data_test_'+str(i)+'.jpg')
    new_img = np.array(new_img)

    # resize into (32,32,3)
    new_img2 = cv.resize(new_img, (32,32), interpolation=cv.INTER_AREA)
    plt.imshow(new_img2)
    plt.axis('off')
    plt.show()

    # preprocess the image using X_train mean and std
    new_img2 = (new_img2 - mean_pixel) / std_pixel

    # predict the class
    pred = myModel.predict(new_img2)
    class_id = np.argmax(pred)
    print('predicted id   :',class_id)
    print('predicted class:', class_names[class_id])
    print('--------------------------------\n\n')


---

# Congratulation, You've Completed Exercise 4

<p>Copyright &copy;  <a href=https://www.linkedin.com/in/andityaarifianto/>2020 - ADF</a> </p>

![footer](https://i.ibb.co/yX0jfMS/footer2020.png)